# Endpoints em TypeScript

Funções assíncronas: estudo e aplicação das estruturas async e await.

https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Promise#description

O código a seguir é o arquivo completo do endpoint.


In [ ]:
import type { APIRoute } from 'astro';
import { encodeBase32LowerCaseNoPadding } from "@/libs/authn/util";
import { clientRepository, authorizationRequestRepository } from '@/libs';
import { sessionProvider } from '@/libs/authn';

export const GET: APIRoute = async ({ cookies, url, redirect }) => {
  const sessionToken = cookies.get('session')?.value;

  if (!sessionToken) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { session } = await sessionProvider.validateSessionToken(sessionToken);

  if (!session) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { user_id } = session

  const requestData = Object.fromEntries(url.searchParams.entries())

  const { response_type, client_id } = requestData

  const client = clientRepository.get({
    id: client_id
  })

  if (!client) {
    return new Response(null, { status: 400 })
  }

  const { callback_endpoint } = client;

  switch (response_type) {
    case "code": {
      const { code_challenge, code_challenge_method } = requestData

      if (code_challenge_method.toLowerCase() !== "s256") {
        return new Response(null, { status: 400 })
      }

      const code = generateAuthorizationCode()

      authorizationRequestRepository.create({
        code,
        client_id,
        project_id: client.project_id,
        code_challenge,
        user_id
      });

      return redirect(`${callback_endpoint}?code=${code}`);
    }
    default: {
      return new Response(null, { status: 400 })
    }
  }
}

function generateAuthorizationCode() {
  const bytes = new Uint8Array(32);
  crypto.getRandomValues(bytes);

  return encodeBase32LowerCaseNoPadding(bytes)
}

O endpoint acima faz parte de uma etapa do provider de um serviço de autenticação oferecido pelo MVP do Bufunfa, o "logar com o Bufunfa". Ele:
- Recebe a solicitação de autorização do CentoBank.
- Se o usuário não estiver logado, ele redireciona para a tela de login, na qual deve ser logado, e redirecionado de volta para o fluxo de autorização (vejam o comportamento do "Login com Google" para detalhes)
- Gera um código de autenticação (code randomico).
- Armazena no banco de dados, na tabela oauth_authorization_request, o code, client_id, code challenge e o user_id.
- Redireciona o usuário para o callback_endpoint criado para o Cento Bank, passando o code gerado.

Nesse sentido, a missão desse notebook é destrinchar todo esse código, objetivando boa compreensão de rotas feitas com TypeScript.

Os blocos de código que seguem abaixo são explicados linha a linha.

```typescript
import type { APIRoute } from 'astro'; 
import { encodeBase32LowerCaseNoPadding } from "@/libs/authn/util";
import { clientRepository, authorizationRequestRepository } from '@/libs';
import { sessionProvider } from '@/libs/authn';
```

```typescript
export const GET: APIRoute = async ({ cookies, url, redirect }) => {
  const sessionToken = cookies.get('session')?.value;
```

```typescript
  if (!sessionToken) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { session } = await sessionProvider.validateSessionToken(sessionToken);
```

```typescript
  if (!session) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { user_id } = session

  const requestData = Object.fromEntries(url.searchParams.entries())

  const { response_type, client_id } = requestData
```

```typescript
  const client = clientRepository.get({
    id: client_id
  })

  if (!client) {
    return new Response(null, { status: 400 });
  }

  const { callback_endpoint } = client;
```

```typescript
  switch (response_type) {
    case "code": {
      const { code_challenge, code_challenge_method } = requestData

      if (code_challenge_method.toLowerCase() !== "s256") {
        return new Response(null, { status: 400 })
      }

      const code = generateAuthorizationCode()

      authorizationRequestRepository.create({
        code,
        client_id,
        project_id: client.project_id,
        code_challenge,
        user_id
      });

      return redirect(`${callback_endpoint}?code=${code}`);
    }
    default: {
      return new Response(null, { status: 400 })
    }
  }

```

```typescript
function generateAuthorizationCode() {
  const bytes = new Uint8Array(32);
  crypto.getRandomValues(bytes);

  return encodeBase32LowerCaseNoPadding(bytes)
}
```

### precisa adaptar

```javascript
const code_verifier = new TextEncoder().encode(atob(code_verifier_cookies));
```

- `code_verifier_cookies` é uma variável que contém uma string codificada em Base64. Este nome sugere que a string foi obtida a partir de cookies armazenados no navegador.

- `atob(code_verifier_cookies)`:
   - Função `atob`: `atob` é uma função JavaScript que decodifica uma string Base64 para texto ASCII. `atob` significa "ASCII to binary".
   - Processo: A string codificada em Base64 (que está armazenada em `code_verifier_cookies`) é decodificada para obter o texto original. 
   - Resultado: O resultado de `atob(code_verifier_cookies)` é uma string em texto plano.

- `new TextEncoder().encode(...)`:
   - `TextEncoder`: `TextEncoder` é uma interface da API de Codificação de Texto que permite converter uma string de texto em um buffer de dados binários codificado em UTF-8.
   - Método `encode`: O método `encode` da `TextEncoder` converte uma string de texto para um `Uint8Array` de bytes.
   - Processo: A string de texto obtida da decodificação Base64 é convertida em uma sequência de bytes codificada em UTF-8.

- Atribuição à Constante `code_verifier`:
   - O resultado final da codificação UTF-8 (um `Uint8Array` de bytes) é armazenado na constante `code_verifier`.